In [1]:
%env CUDA_VISIBLE_DEVICES=0
%env TRANSFORMERS_CACHE=/mnt/LLM/hub
%env OMP_NUM_THREADS=16

import os
import sys
sys.path.insert(0, '..')

import time
import random
from tqdm.auto import trange
import ipynbname  # pip install ipynbname

import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers

from src.aq import QuantizedWeight, _reconstruct_weight  # see adjacent file (aq.py)
from src.utils import calc_avg_bits, get_mean_nbits_by_codebook  # see adjacent file (aq.py)

torch.set_num_threads(16)
torch.backends.cudnn.allow_tf32 = False
torch.backends.cuda.matmul.allow_tf32 = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_loading_dir = '/extra_disk_1/vahe1994/BRRR/layer10.self_attn.q_proj.input_activation.pt'
num_codebooks = 1
nbits_per_codebook = 14
out_group_size = 1
in_group_size = 8
batch_size = 16384
beam_size = 1
beam_search_epochs = 100
sparsity_regularizer = 0
print_frequency = 10
scale_nbits = 2    # 0 means no scales, 16 means no compression;


#for each group we store num_codebooks * nbits_per_codebook - bits 
# for W matrix  we store out_features*(in_features // group_size) * num_codebooks * nbits_per_codebook - bits 
# 1 codebook store codebook_size*group_size*16 - bit 
# all codebooks store num_codebooks* codebook_size*group_size*16 -bits
in_features, out_features  = 8192, 8192
estimated_bits_per_param = calc_avg_bits(
    num_codebooks, out_group_size, in_group_size, nbits_per_codebook, in_features, out_features, scale_nbits)
print("Estimated bits / param", estimated_bits_per_param)

env: CUDA_VISIBLE_DEVICES=0
env: TRANSFORMERS_CACHE=/mnt/LLM/hub
env: OMP_NUM_THREADS=16


/home/jheuristic/anaconda3/lib/python3.9/site-packages/torch/jit/annotations.py:386: UserWarning: TorchScript will treat type annotations of Tensor dtype-specific subtypes as if they are normal Tensors. dtype constraints are not enforced in compilation either.
  warnings.warn(


Estimated bits / param 2.0390625


In [2]:
import wandb

os.environ["WANDB_NOTEBOOK_NAME"] = os.path.join(os.getcwd(), ipynbname.name() + ".ipynb")

# start a new wandb run to track this script
run = wandb.init(
    # set the wandb project where this run will be logged
    dir=os.getcwd(),
    project="AddQuantization",
    entity = "rock-and-roll",
    save_code=True,
    name = f"{ipynbname.name()}_AQ_{num_codebooks=}_{out_group_size=}_{in_group_size=}_{nbits_per_codebook=}_{beam_search_epochs=}",
    settings=wandb.Settings(code_dir="."),
    # track hyperparameters and run metadata
    config={
    "num_codebooks" : num_codebooks,
    "out_group_size": out_group_size,
    "in_group_size": in_group_size,
    "group_size" : out_group_size * in_group_size,
    "batch_size" : batch_size,
    "beam_size" : beam_size,
    "nbits_per_codebook" : nbits_per_codebook,
    "Avg_bits": estimated_bits_per_param,
    "beam_search_epochs": beam_search_epochs,
    "sparsity_regularizer": sparsity_regularizer,
    "scale_nvits": scale_nbits,
    }
)
run.log({"Avg_bits": estimated_bits_per_param})

wandb: Currently logged in as: justheuristic (rock-and-roll). Use `wandb login --relogin` to force relogin


wandb: WARNING No relevant files were detected in the specified directory. No code will be logged to your run.


In [3]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-70b-hf", torch_dtype='auto', low_cpu_mem_usage=True)

X = torch.load(input_loading_dir, map_location='cpu').float().flatten(0, -2)
reference_weight = model.model.layers[10].self_attn.q_proj.weight.detach().to(device).float()

XTX = torch.zeros(X.shape[-1], X.shape[-1], device=device, dtype=torch.float64)
for i in range(0, len(X), batch_size):
    x_batch = X[i: i + batch_size].cuda().double()
    XTX.addmm_(x_batch.T, x_batch, alpha=1/len(X))
    del x_batch
XTX = XTX.float()
del X

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
quantized_weight = QuantizedWeight(
    reference_weight=reference_weight, num_codebooks=num_codebooks,
    nbits_per_codebook=nbits_per_codebook, scale_nbits=scale_nbits,
    out_group_size=out_group_size, in_group_size=in_group_size,
    verbose=True, max_iter=100,   # faster init, not tested
)
opt = torch.optim.Adam(quantized_weight.parameters(), lr=1e-4, betas=(0.0, 0.95), amsgrad=True)

for epoch in range(10_000):
    start = time.perf_counter()
    delta_weight = (quantized_weight() - reference_weight).double()
    loss = (delta_weight @ XTX.double()).flatten() @ delta_weight.flatten() / len(delta_weight)
    opt.zero_grad()
    loss.backward()
    opt.step()
    
    run.log({'loss':loss.item()}, step=epoch)
    
    if epoch % print_frequency == 0:
        print(f"loss={loss.item():.10f}\t",
              f"time_on_epoch {epoch} = {time.perf_counter() - start}")
    if (epoch + 1) % beam_search_epochs == 0:
        quantized_weight.requantize_(
            XTX, reference_weight, beam_size=beam_size, sparsity_regularizer=sparsity_regularizer,
            dim_rng=random.Random(), verbose=True)

        if sparsity_regularizer != 0:
            sparsity_rate = ((quantized_weight.codes == 0).sum() / quantized_weight.codes.numel()).item()
            print(f"Sparsity rate {sparsity_rate:.5f}")
            run.log({'sparsity rate': sparsity_rate}, step=epoch)
            mean_code_nbits = sum(get_mean_nbits_by_codebook(quantized_weight.codes)) / num_codebooks
            print(f"mean_code_nbits {mean_code_nbits:.5f}")
            run.log({'Mean codebook length nbits': mean_code_nbits}, step=epoch)
            if in_group_size > 1 and out_group_size > 1:
                curr_avg_bits  = calc_avg_bits(num_codebooks, 1, mean_code_nbits,
                                     nbits_per_codebook, in_features, out_features, scale_nbits)
                run.log({"Avg_bits": curr_avg_bits}, step=epoch)

/home/jheuristic/GPTAQ_scales/Notebooks/../src/kmeans_1d.py:160: UserWarning: torch.searchsorted(): boundary tensor is non-contiguous, this will lower the performance due to extra data copy when converting non-contiguous tensor to contiguous, please use contiguous boundary tensor if possible. This message will only appear once per program. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/BucketizationUtils.h:38.)
  groupwise_cluster_indices = torch.searchsorted(border_indices[:, 1:], groupwise_ranks_1based, side='left')


initializing with kmeans:   0%|          | 0/1 [00:00<?, ?it/s]

/home/jheuristic/GPTAQ_scales/Notebooks/../src/aq.py:469: UserWarning: index_reduce() is in beta and the API may change at any time. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/cuda/Indexing.cu:1193.)
  codebook_i, codes_i, reconstructed_weight_i = fit_kmeans(weight_residue, k=codebook_size, **kwargs)


loss=0.0254987214	 time_on_epoch 0 = 1.7947464908938855
loss=0.0210565806	 time_on_epoch 10 = 0.13326231797691435
loss=0.0176196967	 time_on_epoch 20 = 0.13337045803200454
loss=0.0149771592	 time_on_epoch 30 = 0.13326312799472362
loss=0.0129895683	 time_on_epoch 40 = 0.1333920379402116
loss=0.0115197716	 time_on_epoch 50 = 0.1334794779540971
loss=0.0104337111	 time_on_epoch 60 = 0.13335301901679486
loss=0.0096248042	 time_on_epoch 70 = 0.1333964589284733
loss=0.0090151296	 time_on_epoch 80 = 0.13328176795039326
loss=0.0085493265	 time_on_epoch 90 = 0.13328312791418284


  0%|          | 0/1024 [00:00<?, ?it/s]

...

  0%|          | 0/1024 [00:00<?, ?it/s]

loss=0.0027642744	 time_on_epoch 200 = 0.13387152599170804
loss=0.0027582571	 time_on_epoch 210 = 0.13328509696293622
loss=0.0027551184	 time_on_epoch 220 = 0.13363595702685416
loss=0.0027528269	 time_on_epoch 230 = 0.13330976699944586
loss=0.0027509907	 time_on_epoch 240 = 0.13335649692453444
loss=0.0027494377	 time_on_epoch 250 = 0.13358324696309865
loss=0.0027480773	 time_on_epoch 260 = 0.1334011269500479
loss=0.0027468570	 time_on_epoch 270 = 0.13318327709566802
loss=0.0027457438	 time_on_epoch 280 = 0.13323979801498353
loss=0.0027447160	 time_on_epoch 290 = 0.13315721694380045


  0%|          | 0/1024 [00:00<?, ?it/s]

loss=0.0025278894	 time_on_epoch 300 = 0.13378514000214636
loss=0.0025244351	 time_on_epoch 310 = 0.13321394904050976
loss=0.0025225416	 time_on_epoch 320 = 0.13309413997922093
loss=0.0025211245	 time_on_epoch 330 = 0.1332467800239101
loss=0.0025199700	 time_on_epoch 340 = 0.1332104590255767
loss=0.0025189815	 time_on_epoch 350 = 0.13313392898999155
loss=0.0025181076	 time_on_epoch 360 = 0.13321038999129087
loss=0.0025173176	 time_on_epoch 370 = 0.1331990489270538
loss=0.0025165923	 time_on_epoch 380 = 0.13323913898784667
loss=0.0025159188	 time_on_epoch 390 = 0.13317411008756608


  0%|          | 0/1024 [00:00<?, ?it/s]